# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f585e101ac0>
├── 'a' --> tensor([[ 0.2580, -1.7902, -0.4562],
│                   [ 0.5850,  0.1944, -0.2701]])
└── 'x' --> <FastTreeValue 0x7f585c058bb0>
    └── 'c' --> tensor([[ 1.3129, -0.0893, -0.8521,  0.6730],
                        [-0.5468,  1.7223, -0.5379, -0.8544],
                        [ 0.4697, -0.4560, -0.7825,  0.4537]])

In [4]:
t.a

tensor([[ 0.2580, -1.7902, -0.4562],
        [ 0.5850,  0.1944, -0.2701]])

In [5]:
%timeit t.a

66.8 ns ± 0.0506 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f585e101ac0>
├── 'a' --> tensor([[ 0.2522,  0.2324, -1.4216],
│                   [-1.1955, -2.7826, -0.4078]])
└── 'x' --> <FastTreeValue 0x7f585c058bb0>
    └── 'c' --> tensor([[ 1.3129, -0.0893, -0.8521,  0.6730],
                        [-0.5468,  1.7223, -0.5379, -0.8544],
                        [ 0.4697, -0.4560, -0.7825,  0.4537]])

In [7]:
%timeit t.a = new_value

69.9 ns ± 0.0527 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.2580, -1.7902, -0.4562],
               [ 0.5850,  0.1944, -0.2701]]),
    x: Batch(
           c: tensor([[ 1.3129, -0.0893, -0.8521,  0.6730],
                      [-0.5468,  1.7223, -0.5379, -0.8544],
                      [ 0.4697, -0.4560, -0.7825,  0.4537]]),
       ),
)

In [10]:
b.a

tensor([[ 0.2580, -1.7902, -0.4562],
        [ 0.5850,  0.1944, -0.2701]])

In [11]:
%timeit b.a

58.6 ns ± 0.124 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-3.3505,  0.5985,  1.3312],
               [-0.9438, -0.8733, -1.4499]]),
    x: Batch(
           c: tensor([[ 1.3129, -0.0893, -0.8521,  0.6730],
                      [-0.5468,  1.7223, -0.5379, -0.8544],
                      [ 0.4697, -0.4560, -0.7825,  0.4537]]),
       ),
)

In [13]:
%timeit b.a = new_value

494 ns ± 0.113 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

817 ns ± 0.176 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.7 µs ± 54.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

144 µs ± 286 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

143 µs ± 546 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f5783dfdf40>
├── 'a' --> tensor([[[ 0.2580, -1.7902, -0.4562],
│                    [ 0.5850,  0.1944, -0.2701]],
│           
│                   [[ 0.2580, -1.7902, -0.4562],
│                    [ 0.5850,  0.1944, -0.2701]],
│           
│                   [[ 0.2580, -1.7902, -0.4562],
│                    [ 0.5850,  0.1944, -0.2701]],
│           
│                   [[ 0.2580, -1.7902, -0.4562],
│                    [ 0.5850,  0.1944, -0.2701]],
│           
│                   [[ 0.2580, -1.7902, -0.4562],
│                    [ 0.5850,  0.1944, -0.2701]],
│           
│                   [[ 0.2580, -1.7902, -0.4562],
│                    [ 0.5850,  0.1944, -0.2701]],
│           
│                   [[ 0.2580, -1.7902, -0.4562],
│                    [ 0.5850,  0.1944, -0.2701]],
│           
│                   [[ 0.2580, -1.7902, -0.4562],
│                    [ 0.5850,  0.1944, -0.2701]]])
└── 'x' --> <FastTreeValue 0x7f5796437fa0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33.1 µs ± 67.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f585c073640>
├── 'a' --> tensor([[ 0.2580, -1.7902, -0.4562],
│                   [ 0.5850,  0.1944, -0.2701],
│                   [ 0.2580, -1.7902, -0.4562],
│                   [ 0.5850,  0.1944, -0.2701],
│                   [ 0.2580, -1.7902, -0.4562],
│                   [ 0.5850,  0.1944, -0.2701],
│                   [ 0.2580, -1.7902, -0.4562],
│                   [ 0.5850,  0.1944, -0.2701],
│                   [ 0.2580, -1.7902, -0.4562],
│                   [ 0.5850,  0.1944, -0.2701],
│                   [ 0.2580, -1.7902, -0.4562],
│                   [ 0.5850,  0.1944, -0.2701],
│                   [ 0.2580, -1.7902, -0.4562],
│                   [ 0.5850,  0.1944, -0.2701],
│                   [ 0.2580, -1.7902, -0.4562],
│                   [ 0.5850,  0.1944, -0.2701]])
└── 'x' --> <FastTreeValue 0x7f57964d37f0>
    └── 'c' --> tensor([[ 1.3129, -0.0893, -0.8521,  0.6730],
                        [-0.5468,  1.7223, -0.5379, -0.8544],
                 

In [23]:
%timeit t_cat(trees)

30.8 µs ± 39.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

66.7 µs ± 102 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.2580, -1.7902, -0.4562],
                [ 0.5850,  0.1944, -0.2701]],
       
               [[ 0.2580, -1.7902, -0.4562],
                [ 0.5850,  0.1944, -0.2701]],
       
               [[ 0.2580, -1.7902, -0.4562],
                [ 0.5850,  0.1944, -0.2701]],
       
               [[ 0.2580, -1.7902, -0.4562],
                [ 0.5850,  0.1944, -0.2701]],
       
               [[ 0.2580, -1.7902, -0.4562],
                [ 0.5850,  0.1944, -0.2701]],
       
               [[ 0.2580, -1.7902, -0.4562],
                [ 0.5850,  0.1944, -0.2701]],
       
               [[ 0.2580, -1.7902, -0.4562],
                [ 0.5850,  0.1944, -0.2701]],
       
               [[ 0.2580, -1.7902, -0.4562],
                [ 0.5850,  0.1944, -0.2701]]]),
    x: Batch(
           c: tensor([[[ 1.3129, -0.0893, -0.8521,  0.6730],
                       [-0.5468,  1.7223, -0.5379, -0.8544],
                       [ 0.4697, -0.4560, -0.7825,  0.4537]],
         

In [26]:
%timeit Batch.stack(batches)

78.5 µs ± 156 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.2580, -1.7902, -0.4562],
               [ 0.5850,  0.1944, -0.2701],
               [ 0.2580, -1.7902, -0.4562],
               [ 0.5850,  0.1944, -0.2701],
               [ 0.2580, -1.7902, -0.4562],
               [ 0.5850,  0.1944, -0.2701],
               [ 0.2580, -1.7902, -0.4562],
               [ 0.5850,  0.1944, -0.2701],
               [ 0.2580, -1.7902, -0.4562],
               [ 0.5850,  0.1944, -0.2701],
               [ 0.2580, -1.7902, -0.4562],
               [ 0.5850,  0.1944, -0.2701],
               [ 0.2580, -1.7902, -0.4562],
               [ 0.5850,  0.1944, -0.2701],
               [ 0.2580, -1.7902, -0.4562],
               [ 0.5850,  0.1944, -0.2701]]),
    x: Batch(
           c: tensor([[ 1.3129, -0.0893, -0.8521,  0.6730],
                      [-0.5468,  1.7223, -0.5379, -0.8544],
                      [ 0.4697, -0.4560, -0.7825,  0.4537],
                      [ 1.3129, -0.0893, -0.8521,  0.6730],
                      [-0.5468,  

In [28]:
%timeit Batch.cat(batches)

146 µs ± 232 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

348 µs ± 4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
